# Yelp Data EDA using PySpark

In [1]:
from pyspark.sql import SparkSession
import utils.config as config
from pyspark.sql.functions import count, avg, explode, split, desc, corr, to_date, year, col

spark = SparkSession.builder.appName('BusinessSimilarityModel').getOrCreate()
df = spark.read.json(config.PHILADELPHIA)
user = spark.read.json(config.USER)

### 1. How many distinct businesses are reviewed in Philadelphia?

In [2]:
df.select('business_id').distinct().count()

5854

### 2. What are the top 10 most reviewed businesses?

In [3]:
df.groupBy("business_id", "name") \
    .agg(count("review_id").alias("num_reviews")) \
    .orderBy(desc("num_reviews")) \
    .show(10, truncate=False)

+----------------------+-----------------------+-----------+
|business_id           |name                   |num_reviews|
+----------------------+-----------------------+-----------+
|ytynqOUb3hjKeJfRj5Tshw|Reading Terminal Market|5778       |
|PP3BBaVxZLcJU54uP_wL6Q|Pat's King of Steaks   |4293       |
|IkY2ticzHEn4QFn8hQLSWg|Geno's Steaks          |3428       |
|9PZxjhTIU7OgPIzuGi89Ew|El Vez                 |3264       |
|ctHjyadbDQAtUFfkcAFEHw|Zahav                  |3173       |
|6ajnOk0GcY9xbb5Ocaw8Gw|Barbuzzo               |2974       |
|j-qtdD55OLfSqfsWuQTDJg|Parc                   |2884       |
|AGlh4ZDv6jnoiYfz7At9mw|Dim Sum Garden         |2778       |
|sTPueJEwcRDj7ZJmG7okYA|Jim's South St         |2769       |
|0RuvlgTnKFbX3IK0ZOOocA|Green Eggs Café        |2733       |
+----------------------+-----------------------+-----------+
only showing top 10 rows



### 3. What are the most common business categories?

In [4]:
df.withColumn("category", explode(split("categories", ","))) \
    .groupBy("category") \
    .count() \
    .orderBy(desc("count")) \
    .show(10)

+--------------------+------+
|            category| count|
+--------------------+------+
|         Restaurants|519950|
|                Food|182647|
|                Bars|181890|
|           Nightlife|175665|
|         Restaurants|167339|
|      American (New)|129942|
|  Breakfast & Brunch|102505|
| American (Tradit...| 96529|
|          Sandwiches| 86698|
|             Italian| 61080|
+--------------------+------+
only showing top 10 rows



### 4. What is the average rating per category?

In [5]:
df.withColumn("category", explode(split("categories", ","))) \
    .groupBy("category") \
    .agg(avg("review_stars").alias("avg_rating")) \
    .orderBy(desc("avg_rating")) \
    .show(10)

+--------------------+-----------------+
|            category|       avg_rating|
+--------------------+-----------------+
| Business Consulting|              5.0|
| Colleges & Unive...|              5.0|
| Party Equipment ...|              5.0|
|  Real Estate Agents|              5.0|
|         Real Estate|              5.0|
| Property Management|              5.0|
|         Hobby Shops|            4.875|
|       Arts & Crafts|4.829457364341085|
|     Airport Lounges|              4.8|
|            Kombucha|4.761904761904762|
+--------------------+-----------------+
only showing top 10 rows



### 5. Who are the most active reviewers?

In [6]:
user.select("user_id", "name", "review_count") \
    .orderBy(desc("review_count")) \
    .show(10, truncate=False)

+----------------------+-------+------------+
|user_id               |name   |review_count|
+----------------------+-------+------------+
|Hi10sGSZNxQH3NLyWSZ1oA|Fox    |17473       |
|hWDybu_KvYLSdEFzGrniTw|Bruce  |16567       |
|P5bUL3Engv-2z6kKohB6qQ|Kim    |9941        |
|nmdkHL2JKFx55T3nq5VziA|Nijole |8363        |
|bQCHF5rn5lMI9c5kEwCaNA|Vincent|8354        |
|8RcEwGrFIgkt9WQ35E6SnQ|George |7738        |
|HFECrzYDpgbS5EmTBtj2zQ|Eric   |5887        |
|m07sy7eLtOjVdZ8oN9JKag|Ed     |5800        |
|kS1MQHYwIfD0462PE61IBw|Rob    |5511        |
|IlGYj_XAMG3v75rfmtBs_Q|Dominik|5434        |
+----------------------+-------+------------+
only showing top 10 rows



### 6. What is the distribution of review ratings?

In [7]:
df.groupBy("review_stars").count().orderBy("review_stars").show()

+------------+------+
|review_stars| count|
+------------+------+
|         1.0| 66626|
|         2.0| 57480|
|         3.0| 91706|
|         4.0|194373|
|         5.0|277122|
+------------+------+



### 7. Correlation between review ratings and votes?

In [8]:
from pyspark.sql.functions import corr

df.select(corr("review_stars", "cool").alias("cool_corr"),
          corr("review_stars", "funny").alias("funny_corr"),
          corr("review_stars", "useful").alias("useful_corr")).show()

+-------------------+-------------------+--------------------+
|          cool_corr|         funny_corr|         useful_corr|
+-------------------+-------------------+--------------------+
|0.07108198527851739|-0.0590778043540176|-0.03484123109418975|
+-------------------+-------------------+--------------------+



### 8. Review activity trend over the years?

In [9]:
from pyspark.sql.functions import to_date, year

df.withColumn("date_clean", to_date("date")) \
  .withColumn("year", year("date_clean")) \
  .groupBy("year") \
  .count() \
  .orderBy("year") \
  .show()

+----+-----+
|year|count|
+----+-----+
|2005|  101|
|2006|  773|
|2007| 4115|
|2008|10912|
|2009|18844|
|2010|29391|
|2011|39371|
|2012|44252|
|2013|51597|
|2014|60250|
|2015|68208|
|2016|68748|
|2017|71947|
|2018|73190|
|2019|72182|
|2020|34860|
|2021|36717|
|2022| 1849|
+----+-----+



### 9. What is the average number of fans per user?

In [10]:
user.selectExpr('avg(fans) as avg_fans').show()

+------------------+
|          avg_fans|
+------------------+
|3.5979944348191317|
+------------------+



### 10. Users with most total compliments?

In [11]:
from pyspark.sql.functions import col

compliment_cols = [col(c) for c in user.columns if c.startswith("compliment_")]

user.withColumn("total_compliments", sum(compliment_cols)) \
     .select("user_id", "name", "total_compliments") \
     .orderBy(desc("total_compliments")) \
     .show(10, truncate=False)

+----------------------+--------+-----------------+
|user_id               |name    |total_compliments|
+----------------------+--------+-----------------+
|JjXuiru1_ONzDkYVrHN0aw|Richard |324328           |
|--2vR0DIsmQ6WfcSzKWigw|Harald  |133351           |
|h4oOQdnfjpEHbygEJDsFbg|John    |132032           |
|JRAy4P4op3PCISZaMRA9_w|Carissa |88363            |
|w0Gp4qYFLhB6PbKrPhn6Tw|Pamela  |87324            |
|HH7iiWvBqV-20lA7JlSRWQ|Lyla    |87271            |
|UXbCcmkYGl3DH_Py5UOtbQ|Toni    |82084            |
|Kf5PVQUdDvQYf-MXoeWqZA|Marianne|80410            |
|AbMjnKOwg736fcIu8apuyQ|Michael |73201            |
|0juzbrgcLHqobdK-OZAyiw|Daniel  |63539            |
+----------------------+--------+-----------------+
only showing top 10 rows

